In [1]:
using MotionCaptureJointCalibration
using RigidBodyDynamics
using ValkyrieRobot
using Ipoptimport Random

Create a `Mechanism` (we'll use the NASA Valkyrie robot) and `MechanismState`:

In [2]:
T = Float64
val = Valkyrie()
mechanism = val.mechanism
remove_fixed_tree_joints!(mechanism)
state = MechanismState{T}(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

Let's generate a random calibration problem, as well as its ground truth solution:

In [3]:
using MotionCaptureJointCalibration.SyntheticDataGeneration
bodies_with_markers = findbody.(Ref(mechanism), ["leftFoot", "pelvis"])
Random.seed!(1)
body_weights = Dict(b => rand() for b in bodies_with_markers)
problem, groundtruth = generate_calibration_problem(state, body_weights);

In [4]:
problem

CalibrationProblem{Float64} with:
* 25 poses
* 6 calibration parameters
* 8 markers attached to 2 bodies



In [5]:
groundtruth

CalibrationResult{Float64}: Optimal, residual = 0.0. Calibration parameters:
leftHipRoll: [0.00907607]
leftKneePitch: [0.000393507]
leftAnklePitch: [-0.00972919]
leftHipPitch: [-0.00809629]
leftHipYaw: [0.00473574]
leftAnkleRoll: [-0.00393202]

Solve the calibration problem (without using the ground truth result):

In [6]:
solver = IpoptSolver(print_level = 0, max_iter = 10000, derivative_test = "first-order", check_derivatives_for_naninf = "yes", tol = 1e-10)
result = solve(problem, solver)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



CalibrationResult{Float64}: Optimal, residual = 1.1148801235706582e-9. Calibration parameters:
leftHipRoll: [0.00908942]
leftKneePitch: [0.000380969]
leftAnklePitch: [-0.00981683]
leftHipPitch: [-0.00804019]
leftHipYaw: [0.00472053]
leftAnkleRoll: [-0.00404254]

And visualize the results:

In [7]:
using RigidBodyTreeInspector
using DrakeVisualizer

In [8]:
#NBSKIP
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));

In [9]:
vis = Visualizer()[:valkyrie]
geometry = visual_elements(mechanism, URDFVisuals(ValkyrieRobot.urdfpath(); package_path = [ValkyrieRobot.packagepath()]))
setgeometry!(vis, mechanism, geometry)
state = MechanismState{T}(mechanism)

2018-03-21 12:56:26.433 drake-visualizer[54215:6726934] *** WARNING: Method userSpaceScaleFactor in class NSView is deprecated on 10.7 and later. It should not be used in new applications. Use convertRectToBacking: instead. 


MechanismState{Float64, Float64, Float64, …}(…)

In [10]:
inspect!(state, vis, problem, result)

Interact.Options{:SelectionSlider,Any}(1: "input" = false Any , "", false, "Before cal", 0, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("Before cal"=>false,"After cal"=>true), Dict{Any,Any}(Pair{Any,Any}(false, "Before cal"),Pair{Any,Any}(true, "After cal"))), Any[], Any[], true, "horizontal")

Interact.Slider{Int64}(3: "input-2" = 1 Int64 , "Pose number", 1, 1:25, "horizontal", true, "d", true)